In [1]:
VERSION

v"1.8.1"

In [2]:
ENV["COLUMNS"] = 1000
ENV["LINES"] = 20;

In [6]:
# using Pkg
# Pkg.add("GLMakie")
# Pkg.update("GLMakie")
# Pkg.build("GLMakie")

In [8]:
using DataFrames
using GLMakie

using Revise
using Astroshaper
import SPICE

In [9]:
meta_kernel = "/Users/masanorikanamaru/Dropbox/spice/hera/kernels/mk/hera_study_PO_EMA_2024_shapeViewer.tm"

SPICE.furnsh(meta_kernel)

In [10]:
et_start = SPICE.utc2et("2027-02-09T22:00:00")
et_end   = SPICE.utc2et("2027-02-11T22:00:00")
step     = 3600/20

et_range = et_start:step:et_end

8.554824691849957e8:180.0:8.556552691849957e8

In [11]:
"""
Obtain a dataframe of ephemerides 

# Arguments
- `targ`   : Target body name
- `ets`    : AbstractVetor of observer epoch
- `ref`    : Reference frame of output position vector
- `abcorr` : Aberration correction flag
- `obs`    : Observing body name
"""
function spkpos_df(targ, ets::AbstractVector, ref, abcorr, obs)
    df = DataFrame(et=Float64[], x=Float64[], y=Float64[], z=Float64[], lt=Float64[])
    for et in ets
        pos, lt = SPICE.spkpos(targ, et, ref, abcorr, obs)  # pos [km], lt [s]
        pos = SPICE.convrt.(pos, "km", "m")
        push!(df, (et, pos..., lt))
    end
    df
end

function spkpos_vec(targ, ets::AbstractVector, ref, abcorr, obs)
    positions = Vector{Float64}[]
    for et in ets
        pos, lt = SPICE.spkpos(targ, et, ref, abcorr, obs)  # pos [km], lt [s]
        pos = SPICE.convrt.(pos, "km", "m")
        push!(positions, collect(pos))
    end
    positions
end


"""
# Arguments
- `from` : Name of the frame to transform from
- `to`   : Name of the frame to transform to
- `ets`  : Epoch of the rotation matrix

# Return
- `rotate` : A rotation matrix
"""
pxform(from, to, ets) = [SPICE.pxform(from, to, et) for et in ets];

In [12]:
sun_d1 = spkpos_vec("SUN",       et_range, "DIDYMOS_FIXED",   "None", "DIDYMOS")
sun_d2 = spkpos_vec("SUN",       et_range, "DIMORPHOS_FIXED", "None", "DIMORPHOS")
d1_d2  = spkpos_vec("DIDYMOS",   et_range, "DIMORPHOS_FIXED", "None", "DIMORPHOS")
d2_d1  = spkpos_vec("DIMORPHOS", et_range, "DIDYMOS_FIXED",   "None", "DIDYMOS");

In [13]:
D1_TO_D2 = pxform("DIDYMOS_FIXED",   "DIMORPHOS_FIXED", et_range)
D2_TO_D1 = pxform("DIMORPHOS_FIXED", "DIDYMOS_FIXED",   et_range);

In [14]:
SPICE.kclear()

In [17]:
fig = Figure()
ax = Axis3(fig[1, 1], aspect=:data)

# scatter!(0,0,0)
# scatter!(sun_d1.x, sun_d1.y, sun_d1.z, color=:orange)
# scatter!(d2_d1.x, d2_d1.y, d2_d1.z, color=:skyblue)
# scatter!(sun_d1.x, sun_d1.y, sun_d1.z, color=:orange)
# scatter!(d2_d1.x, d2_d1.y, d2_d1.z, color=:skyblue)

scatter!([r[1] for r in sun_d1], [r[2] for r in sun_d1], [r[3] for r in sun_d1], color=:orange, size=2)

display(fig)

GLMakie.Screen(...)

- まずはDidymosだけで熱計算してみる
- まずはDimorphosだけで熱計算してみる
- 二重小惑星として熱計算してみる

In [53]:
shapedir = "/Users/masanorikanamaru/Dropbox/spice/hera/kernels/dsk"

shapepath1 = joinpath(shapedir, "g_50677mm_rad_obj_dida_0000n00000_v001.obj")
shapepath2 = joinpath(shapedir, "g_06650mm_rad_obj_didb_0000n00000_v001.obj")

shape1 = ShapeModel(shapepath1; scale=1000, find_visible_facets=true, save_shape=false)
shape2 = ShapeModel(shapepath2; scale=1000, find_visible_facets=true, save_shape=false);

# Only Dimorphos

In [49]:
thermo_params = ThermoParams(  # [Michel+2016; Naidu+2020]
    A_B   = 0.059,  # Bolometric  Bond albedo
    A_TH  = 0.0,
    k     = 0.125,
    ρ     = 2170.,
    Cp    = 600.,
    ϵ     = 0.9,
    t_bgn = et_range[begin],
    t_end = et_range[end],
    Nt    = length(et_range),
    z_max = 0.6,
    Nz    = 41,
    P     = SPICE.convrt(DIDYMOS[:P], "hours", "seconds"),
)

Thermophysical parameters
-------------------------
A_B   : 0.059
A_TH  : 0.0
k     : 0.125
ρ     : 2170.0
Cp    : 600.0
ϵ     : 0.9
-------------------------
t_bgn : 8.554824691849957e8
t_bgn : 105180.37410616312 (Normalized by period P)
t_end : 8.556552691849957e8
t_end : 105201.61962468656 (Normalized by period P)
Nt    : 961
Δt    : 180.0000000000556
Δt    : 0.022130748461919817 (Normalized by period P)
-------------------------
z_max : 0.6
z_max : 6.057014348123621 (Normalized by skin depth l)
Nz    : 41
Δz    : 0.015
Δz    : 0.15142535870309054 (Normalized by skin depth l)
-------------------------
P     : 8133.4800000000005
l     : 0.09905870541414048
Γ     : 403.4228550788862
λ     : 0.07680491551461667
-------------------------


In [60]:
timestamp = run_TPM(shape1, et_range, sun_d1, thermo_params, "tmp.jld2");